In [ ]:
%pip install -r requirements.txt
# %pip install nitrado "steam[client]" requests python-dotenv
%config IPCompleter.greedy=True

**1> collate workshop_id to mod_id(s) from source of truth collection**

In [ ]:
# Find and collate workshop_ids: mod_ids, store 
#data in an output var
import requests
import re
from os import path

collection_id = '2903054110'
getCollections = 'https://api.steampowered.com/ISteamRemoteStorage/GetCollectionDetails/v1/?'
getFileDetails = 'https://api.steampowered.com/ISteamRemoteStorage/GetPublishedFileDetails/v1/?'


home_dir = path.expanduser('~')
print(f'user home dir:{home_dir}')

# output contains the resulting list of tuples [(workshop_id, mod_id, mod_id, ...), ...] produced
# in this sell.
output = list()

# collection_data is a locally consumed var and contains the post-processed collection metadata response
collection_data = list()
with open(path.join(home_dir, '.steam', 'token'), 'r') as f:
    auth_header = {'Authentication': f'Bearer {f.read().strip()}'}
    resp = requests.post(
        getCollections, 
        headers=auth_header,
        data={
            'publishedfileids[0]':f'{collection_id}',
            'collectioncount': '1',
        }
    )
    collection_data = resp.json()['response']['collectiondetails']


for child in collection_data[0]['children']:
    child_id = child["publishedfileid"]
    resp = requests.post(
        getFileDetails,
        headers=auth_header,
        data={
            'itemcount': '1',
            'publishedfileids[0]': child_id,
        }
    )
    desc = resp.json()['response']['publishedfiledetails'][0]['description']
    match = re.findall(r'\nMod ID: *(\w+)', desc)
    if match is None:
        print(f'failed to find matching mod id string, mod file: {child_id}')
        quit(1)

    o = [child_id]
    for m in match:
        o.append(m)
    print(o)
    output.append(tuple(o))

**^^TODO create radial buttons per mod, per variant**

**2> Fetch the server config file**

In [ ]:
from os import path
from ftplib import FTP
from configparser import ConfigParser, MissingSectionHeaderError
import io
import re

nitrado_dir = path.join(home_dir, ".nitrado")
nitrado_ftp_url = 'usmi440.gamedata.io'
ftp_config = ConfigParser()
ftp_config.read(path.join(nitrado_dir, "ftp.ini"))
section = 'dummy'


pz_cfg_src = path.join('/zomboid/profile/Zomboid/Server/servertest.ini')
pz_config = ConfigParser(allow_no_value=True)

buf = io.BytesIO()
buf.write(f'[{section}]\n'.encode())
with FTP(nitrado_ftp_url) as ftp:
    ftp.login(
        user = ftp_config['DEFAULT']['username'], 
        passwd = ftp_config['DEFAULT']['password']
    )
    ftp.retrbinary(f'RETR {pz_cfg_src}', buf.write)
    buf.seek(0, io.SEEK_SET)

# Remember to remove the dummy section header before writing file to ftp
pz_config.read_string(buf.read().decode())

**3> Merge the collection into the pz_config vars**

In [ ]:
mod_list_sep = ','
ws_item_sep = ';'


merge_mods = pz_config[f'{section}']['mods'].split(mod_list_sep)
merge_ws_items = pz_config[f'{section}']['workshopitems'].split(ws_item_sep)

# Add new mods (i.e. mods not found in the config)
for o in output:
    ws_item = o[0]
    mods = o[1:]
    print(f'processing: {ws_item}: {mods}')
    if ws_item not in merge_ws_items:
        print(f"{ws_item}({mods[0]}) not found, appending")
        merge_ws_items.append(ws_item)
        merge_mods.append(mods[0])
    else:
        continue
        print(f'{ws_item} already added, skipping')
        
# Remove old mods (i.e. mods not found in the collection)
for i in range(len(merge_ws_items) - 1):
    if i < 0:
        print('got zero len list')
        quit()
    # TODO this should not default to first mod variant.
    if merge_ws_items[i] not in [o[0] for o in output]:
        print(f'workshopitem {merge_ws_items[i]}({merge_mods[i]}) not found in existing config, removing')
        del merge_ws_items[i]
        del merge_mods[i]
        
if len(merge_mods) is not len(merge_ws_items):
    print(f'something went wrong, len(merge_mods)[{len(merge_mods)}] != len(merge_ws_items)[{len(merge_ws_items)}]')
    quit(1)

# We're ready to go

In [ ]:
DEBUG = True

**4> Write the pz config back to the ftp**

In [ ]:
import tempfile
from io import SEEK_SET
import os

# skip the hacky section header
buf.seek(0, SEEK_SET)
buf.readline()

new_mods = mod_list_sep.join(merge_mods)
new_ws_ids = ws_item_sep.join(merge_ws_items)
if DEBUG:
    print(f'new_mods={new_mods}')
    print(f'new_ws={new_ws_ids}')
    print('----------------------')

out_cfg = re.sub(fr'\nMods=[a-zA-Z0-9{mod_list_sep}]*', repl=f'\nMods={new_mods}', string=buf.read().decode())
out_cfg = re.sub(fr'\nWorkshopItems=[0-9{ws_item_sep}]*', repl=f'\nWorkshopItems={new_ws_ids}', string=out_cfg)
if DEBUG:
    print(out_cfg)
    print("======================")

    print_m = re.search(fr'(\nMods=[a-zA-Z0-9{mod_list_sep}]*\n)', out_cfg)
    if print_m is not None:
        print(print_m.group(0))
    print_w = re.search(fr'\nWorkshopItems=[a-zA-Z0-9{ws_item_sep}]*\n', out_cfg)
    
    print(print_w.group(0))

# TODO fix the FTP upload path
with FTP(nitrado_ftp_url) as ftp:
    ftp.login(
        user = ftp_config['DEFAULT']['username'], 
        passwd = ftp_config['DEFAULT']['password']
    )
    out_buf = io.StringIO()
    out_buf.write(out_cfg)
    out_buf.seek(0, SEEK_SET)
    with open(os.path.join(home_dir, 'servertest.ini'), 'w') as f:
        if DEBUG:
            f.write(out_buf.getvalue())
        else:
            ftp.storlines(cmd=f'STOR {pz_cfg_src}', fp=io.BytesIO(out_buf.getvalue().encode()))